<a href="https://colab.research.google.com/github/Jeeva55t/Large_Language_Model-Projects/blob/main/TinyLlama_powered_Document_QA__System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q langchain llama-cpp-python faiss-cpu sentence-transformers

In [14]:
!wget -q https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf -O tinyllama.gguf

In [17]:
!pip -q install langchain_community

In [6]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp

In [8]:
#step 1 :load and split documents
loader = TextLoader('india.txt')
documents = loader.load()

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

In [9]:
#step 2 :Embed and index
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_DB = FAISS.from_documents(docs, embeddings)

/tmp/ipython-input-9-3887086584.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from re import VERBOSE
#step 3: Load the tinyLLama
llm = LlamaCpp(
    model_path="tinyllama.gguf",
    temperature = 0.5,
    max_tokens =  1024,
    top_p = 0.95,
    n_ctx = 4096, # Increased context window size
    VERBOSE=True
)

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! VERBOSE is not default parameter.
                VERBOSE was transferred to model_kwargs.
                Please confirm that VERBOSE is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from tinyllama.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                 

In [11]:
#step 4: Create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_DB.as_retriever()
)

In [12]:
question = "what is the capital of India?"
answer = qa_chain.run(question)
print(answer)

/tmp/ipython-input-12-2910966417.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(question)
llama_perf_context_print:        load time =   40070.74 ms
llama_perf_context_print: prompt eval time =   40070.44 ms /   595 tokens (   67.35 ms per token,    14.85 tokens per second)
llama_perf_context_print:        eval time =     922.26 ms /     8 runs   (  115.28 ms per token,     8.67 tokens per second)
llama_perf_context_print:       total time =   41003.64 ms /   603 tokens


 Delhi is the capital of India.
